In [1]:
import sys
sys.path.append('../src')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from tip_amount_model import TipAmountModelConfig, TipAmountModel

[2026-01-08 15:02] INFO: /Users/alex/repos/blog_posts/pyspark_to_production/notebooks/../src/tip_amount_model.py logger is initialized!


In [4]:
config = TipAmountModelConfig()
job = TipAmountModel(config)

26/01/08 15:02:30 WARN Utils: Your hostname, Alexandrs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.108 instead (on interface en0)
26/01/08 15:02:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/08 15:02:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
job.run()

[2026-01-08 15:02] INFO: Extracting datasets
[2026-01-08 15:02] INFO: Preparing the data for training                        
[2026-01-08 15:02] INFO: Training the model
[2026-01-08 15:02] INFO: Start validation                                       
[2026-01-08 15:02] INFO:   Features importances
[2026-01-08 15:02] INFO:           payment_type = 0.65
[2026-01-08 15:02] INFO:            fare_amount = 0.22
[2026-01-08 15:02] INFO:          trip_distance = 0.052
[2026-01-08 15:02] INFO:              rate_code = 0.049
[2026-01-08 15:02] INFO:           tolls_amount = 0.028
[2026-01-08 15:02] INFO:           day_of_month = 0.0023
[2026-01-08 15:02] INFO:                  month = 0.0019
[2026-01-08 15:02] INFO:            day_of_week = 0.0017
[2026-01-08 15:02] INFO:          imp_surcharge = 0.00076
[2026-01-08 15:02] INFO:        passenger_count = 0.0007
[2026-01-08 15:02] INFO:     store_and_fwd_flag = 5.1e-05
[2026-01-08 15:02] INFO:   Evaluation on the training set
[2026-01-08 15:02] IN

In [6]:
job.sdfs["training"].show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+-----+-----------+------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|month|day_of_week|day_of_month|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+-----+-----------+------------+
|        1|2018-01-01 16:48:36|2018-01-01 17:20:58|              4|         10.0|        1|                 0|           1|       32.5|  0.0|    0.5|       4.0|         0.0|          0.3|

In [9]:
job.config.test_fraction = 0.01
job.transform()
job.validate()

[2026-01-08 14:12] INFO: Preparing the data for training
[2026-01-08 14:12] INFO: Training the model
[2026-01-08 14:12] INFO: Start validation
[2026-01-08 14:12] INFO:   Features importances
[2026-01-08 14:12] INFO:           payment_type = 0.67
[2026-01-08 14:12] INFO:            fare_amount = 0.21
[2026-01-08 14:12] INFO:              rate_code = 0.051
[2026-01-08 14:12] INFO:          trip_distance = 0.03
[2026-01-08 14:12] INFO:           tolls_amount = 0.03
[2026-01-08 14:12] INFO:        passenger_count = 0.0058
[2026-01-08 14:12] INFO:                  month = 0.002
[2026-01-08 14:12] INFO:          imp_surcharge = 0.0013
[2026-01-08 14:12] INFO:           day_of_month = 0.0011
[2026-01-08 14:12] INFO:            day_of_week = 1.9e-05
[2026-01-08 14:12] INFO:     store_and_fwd_flag = 0
[2026-01-08 14:12] INFO:   Evaluation on the training set
[2026-01-08 14:12] INFO:     rmse = 3.1
[2026-01-08 14:13] INFO:      mae = 1.6
[2026-01-08 14:13] INFO:       r2 = 0.37
[2026-01-08 14:13

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor

def train_model(self) -> None:
    assembler = VectorAssembler(inputCols=self.feature_cols, outputCol="features")

    gbt = GBTRegressor(
        labelCol="tip_amount",
        featuresCol="features",
        predictionCol="prediction",
        stepSize=0.1,
        maxDepth=4,
        featureSubsetStrategy="auto",
        seed=42,
    )

    pipeline = Pipeline(stages=[assembler, gbt])

    self.model = pipeline.fit(self.sdfs["training"])

    print("Modified content")

In [7]:
import types
job.train_model = types.MethodType(train_model, job)

In [8]:
job.transform()
job.validate()

[2026-01-08 15:02] INFO: Preparing the data for training
[2026-01-08 15:02] INFO: Training the model
[2026-01-08 15:03] INFO: Start validation                                       
[2026-01-08 15:03] INFO:   Features importances
[2026-01-08 15:03] INFO:           payment_type = 0.38
[2026-01-08 15:03] INFO:            fare_amount = 0.17
[2026-01-08 15:03] INFO:        passenger_count = 0.098
[2026-01-08 15:03] INFO:          trip_distance = 0.069
[2026-01-08 15:03] INFO:           tolls_amount = 0.066
[2026-01-08 15:03] INFO:              rate_code = 0.06
[2026-01-08 15:03] INFO:                  month = 0.058
[2026-01-08 15:03] INFO:           day_of_month = 0.057
[2026-01-08 15:03] INFO:          imp_surcharge = 0.02
[2026-01-08 15:03] INFO:            day_of_week = 0.014
[2026-01-08 15:03] INFO:     store_and_fwd_flag = 0
[2026-01-08 15:03] INFO:   Evaluation on the training set


Modified content


26/01/08 15:03:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
26/01/08 15:03:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
[2026-01-08 15:03] INFO:     rmse = 2.9                                         
[2026-01-08 15:03] INFO:      mae = 1.4
[2026-01-08 15:03] INFO:       r2 = 0.43
[2026-01-08 15:03] INFO:   Evaluation on the test set
[2026-01-08 15:03] INFO:     rmse = 3.1
[2026-01-08 15:03] INFO:      mae = 1.4
[2026-01-08 15:03] INFO:       r2 = 0.4
